In [2]:
import pandas as pd
import os

# Helpful functions

In [8]:
def filter_for_our_years(df):
    return df[(df['yr_id'] == 2020) | (df['yr_id'] == 2021)]

# Age

In [3]:
geo_level = 'region'

In [16]:
# 2022_01
age_2022_01_file = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Data\age\{geo_level}_age_est_2022_01_ind_QA.csv')
age_2022_01_file = filter_for_our_years(age_2022_01_file)
age_2022_01_file = age_2022_01_file.set_index([geo_level, 'yr_id'])
age_2022_01_file

Under 5  10 to 14  15 to 17  18 and 19  20 to 24  25 to 29  \
region    yr_id                                                               
San Diego 2020    221150    221337    121485     105558    282225    298642   
          2021    209060    215490    117097      98600    283006    289033   

                 30 to 34  35 to 39  40 to 44  45 to 49  5 to 9  50 to 54  \
region    yr_id                                                             
San Diego 2020     271572    235732    209104    203010  203886    201784   
          2021     265311    239966    213336    198370  204741    187093   

                 55 to 59  60 and 61  62 to 64  65 to 69  70 to 74  75 to 79  \
region    yr_id                                                                
San Diego 2020     192300      64243     92766    139543    100341     65090   
          2021     194529      69011     88984    135799    106334     65727   

                 80 to 84  85 and Older  
region    yr_id                          
San Diego 2020      43852         57659  
          2021      46538         60478

In [18]:
# 2021_01
age_2021_01_file = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\age\QA_2021_01_{geo_level}_age.csv')
age_2021_01_file = filter_for_our_years(age_2021_01_file)
age_2021_01_file = age_2021_01_file.set_index([geo_level, 'yr_id'])
age_2021_01_file

Under 5  5 to 9  10 to 14  15 to 17  18 and 19  20 to 24  \
region    yr_id                                                             
San Diego 2020    205904  230989    223104    133490      98931    261861   
          2021    197646  229493    225022    135190      97988    259127   

                 25 to 29  30 to 34  35 to 39  40 to 44  45 to 49  50 to 54  \
region    yr_id                                                               
San Diego 2020     207190    208315    231394    208834    204996    200622   
          2021     205113    202317    224786    211732    199960    200006   

                 55 to 59  60 and 61  62 to 64  65 to 69  70 to 74  75 to 79  \
region    yr_id                                                                
San Diego 2020     207276      80734    114670    164902    134013     88881   
          2021     201821      81186    114951    168295    139549     92723   

                 80 to 84  85 and Older  
region    yr_id                          
San Diego 2020      60482         64691  
          2021      62146         66353

In [83]:
def get_input_data(category, geo_level):
    # 2022_01
    df_2022_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Data\{category}\{geo_level}_{category}_est_2022_01_ind_QA.csv')
    df_2022_01 = filter_for_our_years(df_2022_01)
    df_2022_01 = df_2022_01.set_index([geo_level, 'yr_id'])
    
    if category == 'income_group':
        category = 'household_income'
    # 2021_01 
    df_2021_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_{geo_level}_{category}.csv')
    df_2021_01 = filter_for_our_years(df_2021_01)
    df_2021_01 = df_2021_01.set_index([geo_level, 'yr_id'])
    
    return df_2022_01, df_2021_01

In [43]:
df_2022_01, df_2021_01 = get_input_data(category='age', geo_level='region')

# Functions

In [37]:
import pandas as pd

def subtract_dataframes(df1, df2):
    # Find the common columns between the two data frames
    common_cols = list(set(df1.columns) & set(df2.columns))
    common_cols = [x for x in df2.columns if x in common_cols]
    # Filter the data frames to keep only the common columns
    df1 = df1[common_cols]
    df2 = df2[common_cols]
    
    # Subtract the second data frame from the first
    result = df1.subtract(df2)
    
    return result


In [38]:
def write_to_excel(df_2022, df_2021, category, geo_level):
    diff = subtract_dataframes(df_2022, df_2021)
    # Create a Pandas Excel writer using xlsxwriter as the engine
    writer = pd.ExcelWriter(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\diff_outputs\{category}\{geo_level}_diff_2022_minus_2021.xlsx', engine='xlsxwriter')

    # Write each data frame to a different sheet
    df_2022.to_excel(writer, sheet_name='2022_01 Data', index=True)
    df_2021.to_excel(writer, sheet_name='2021_01 Data', index=True)
    diff.to_excel(writer, sheet_name='Diff', index=True)

    # Save the writer
    writer.save()

# Run

In [139]:
category = 'age_sex_ethnicity'
geo_level = 'cpa'

In [140]:
df_2022_01, df_2021_01 = get_input_data(category=category, geo_level=geo_level)

In [141]:
df_2022_01 = df_2022_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()

In [142]:
df_2021_01 = df_2021_01.reset_index().groupby([geo_level, 'yr_id', 'name', 'sex']).sum()

In [143]:
write_to_excel(df_2022_01, df_2021_01, category, geo_level)